In [8]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp
from scipy.integrate import odeint
import plotly.graph_objects as go
from IPython.display import HTML

In [9]:
t, h, g, m, x0, p, w, J1, J3 = smp.symbols(r't h g m x_0 \Phi \omega J_1, J_3', real=True)
the, phi, psi = smp.symbols(r'\theta \phi \psi', cls=smp.Function)
the = the(t)
phi = phi(t)
psi = psi(t)
# Derivatives
the_d = smp.diff(the,t)
phi_d = smp.diff(phi,t)
psi_d = smp.diff(psi,t)
# Second derivatives
the_dd = smp.diff(the_d,t)
phi_dd = smp.diff(phi_d,t)
psi_dd = smp.diff(psi_d,t)

R3 = smp.Matrix([[smp.cos(psi),-smp.sin(psi),0],
                 [smp.sin(psi),smp.cos(psi),0],
                 [0,0,1]])

R2 = smp.Matrix([[1,0,0],
                 [0,smp.cos(the),-smp.sin(the)],
                 [0,smp.sin(the),smp.cos(the)]])

R1 = smp.Matrix([[smp.cos(phi),-smp.sin(phi),0],
                 [smp.sin(phi),smp.cos(phi),0],
                 [0,0,1]])

R = R1*R2*R3

omega = smp.Matrix([phi_d*smp.sin(the)*smp.sin(psi)+the_d*smp.cos(psi),
                    phi_d*smp.sin(the)*smp.cos(psi)-the_d*smp.sin(psi),
                    phi_d*smp.cos(the)+psi_d])

I = smp.Matrix([[J1,0,0],[0,J1,0],[0,0,J3]])

xc = h * R @ smp.Matrix([0, 0, 1]) + smp.Matrix([x0 * smp.cos(w * t + p), 0, 0])
xc.simplify()
vc = smp.diff(xc, t)
vc.simplify()
vc_carre = vc.T.dot(vc)
vc_carre.simplify()

xc_free = h * R @ smp.Matrix([0, 0, 1])
xc_free.simplify()
vc_free = smp.diff(xc_free, t)
vc_free.simplify()
vc_carre_free = vc_free.T.dot(vc_free)
vc_carre_free.simplify()


# Kinetic, and potential energy
T_rot = (smp.Rational(1, 2) * omega.T.dot(I * omega).simplify())  # Rotational kinetic energy
T_c = smp.Rational(1, 2) * m * vc_carre  # Translational kinetic energy
T_c = T_c.simplify()
T = T_c + T_rot  # Total kinetic energy
V = m * g * h * smp.cos(the)  # Potential energy (gravitation)

T_free = smp.Rational(1, 2) * m * vc_carre_free + T_rot
T_free = T_free.simplify()

# Lagrangian
L = T_free - V
L = L.simplify()
L_f = T - V
L_f = L_f.simplify()

genCoordinates = [the, phi, psi]
genSpeeds = [the_d, phi_d, psi_d]
genAcceleration = [the_dd, phi_dd, psi_dd]

In [12]:
PG = R @ smp.Matrix([0,0, 1])
PG

Matrix([
[ sin(\phi(t))*sin(\theta(t))],
[-sin(\theta(t))*cos(\phi(t))],
[              cos(\theta(t))]])

In [18]:
ae = - (w**2) * x0 * smp.Matrix([smp.cos(w*t+p), smp.sin(w*t+p), 0])
ae

Matrix([
[-\omega**2*x_0*cos(\Phi + \omega*t)],
[-\omega**2*x_0*sin(\Phi + \omega*t)],
[                                  0]])

In [19]:
Fe = -m*ae
Fe

Matrix([
[\omega**2*m*x_0*cos(\Phi + \omega*t)],
[\omega**2*m*x_0*sin(\Phi + \omega*t)],
[                                   0]])

In [20]:
Fe.dot(PG).simplify()

-\omega**2*m*x_0*sin(\Phi + \omega*t - \phi(t))*sin(\theta(t))

In [24]:
k = PG.cross(Fe)
k.simplify()
k

Matrix([
[         -\omega**2*m*x_0*sin(\Phi + \omega*t)*cos(\theta(t))],
[          \omega**2*m*x_0*cos(\Phi + \omega*t)*cos(\theta(t))],
[\omega**2*m*x_0*sin(\theta(t))*cos(\Phi + \omega*t - \phi(t))]])